# DOJO ML - 3rd Place solution
## team : curiosos_do_ml 
## Mateus C. Pedrino - Bruno Rasteiro - HelloToMyLittleFriend

The general goal of this big notebook (merge of each notebook involved in our solution) is to show some baby steps that we used to solve the problem of this competition.

In [42]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

## Preparing Data

### Params

In [3]:
data_type = ['treino', 'teste'][0]
days = '0 days' # time window
window = pd.Timedelta(days)

In [4]:
if data_type == 'treino':
    group_columns = ['maquina', 'data', 'falha']
else:
    group_columns = ['maquina', 'data', 'index']

Once we have an historical data, we created the feature "window" so we are able to group features in "window" days to generate the feature that is impacting the current day.

For instance, imagine we're inputing the training data respective to day 2015-08-07. Sensors data (temperature, pressure, etc) is measured each hour. So what will it be the temperature value in 2015-08-07? We can consider the mean, std, RMS and other grouping operation within "window" days to calculate the temperature value for the day. So, if "window" = 0 and the grouping operation is mean for temperature feature, the value of temperature in 2015-08-07 will be the mean of each hour temperature in day 2015-08-07. If "window" = 3, the value of temperature in 2015-08-07 will be the mean of each hour temperature between days 2015-08-07 and 2015-08-04. We believe that this might help to train the model if the past of the feature impacts the current target variable.

### Load data

In [5]:
root_path = './'

In [6]:
erros = pd.read_csv(root_path + 'erros.csv')
info_uso = pd.read_csv(root_path + 'info_uso.csv')
maquinas = pd.read_csv(root_path + 'maquinas.csv')
dados = pd.read_csv(root_path + '{}.csv'.format(data_type))

#treino = pd.read_csv(root_path+'dojoml-saocarlos/treino.csv')
#teste = pd.read_csv(root_path+'dojoml-saocarlos/teste.csv')

In [7]:
info_uso.head()

,data,maquina,voltagem,rotacao,pressao,vibracao
0,2015-01-01 06:00:00,1,176.217853,418.504078,113.077935,45.087686
1,2015-01-01 07:00:00,1,162.879223,402.747490,95.460525,43.413973
2,2015-01-01 08:00:00,1,170.989902,527.349825,75.237905,34.178847
3,2015-01-01 09:00:00,1,162.462833,346.149335,109.248561,41.122144
4,2015-01-01 10:00:00,1,157.610021,435.376873,111.886648,25.990511


In [8]:
# Cast datetime fields to datetime type
erros['data'] = pd.to_datetime(erros['data'])
info_uso['data'] = pd.to_datetime(info_uso['data'])
dados['data'] = pd.to_datetime(dados['data'])

## Generating database

### Merge dados with maquinas => df_maquinas

In [9]:
dados.head()

,index,maquina,data,falha
0,20012,55,2015-08-10,ok
1,8002,22,2015-10-28,ok
2,6190,17,2015-11-18,ok
3,12536,35,2015-03-12,ok
4,34478,95,2015-02-04,ok


In [10]:
maquinas.head()

,maquina,modelo,idade
0,1,model3,18
1,2,model4,7
2,3,model3,8
3,4,model3,7
4,5,model3,2


We can merge "dados" with "maquinas" using maquina (identifies the machine) so we'll be able to add model (modelo) and age ("idade") information to our training set. 

In [11]:
df_maquinas = pd.merge(dados, maquinas, on='maquina')

In [12]:
df_maquinas.head()

,index,maquina,data,falha,modelo,idade
0,20012,55,2015-08-10,ok,model3,17
1,19938,55,2015-05-28,ok,model3,17
2,19886,55,2015-04-06,ok,model3,17
3,20129,55,2015-12-05,ok,model3,17
4,20101,55,2015-11-07,ok,model3,17


### Merge dados with erros => df_erros

In [13]:
erros.head()

,data,maquina,erro
0,2015-01-03 07:00:00,1,error1
1,2015-01-03 20:00:00,1,error3
2,2015-01-04 06:00:00,1,error5
3,2015-01-10 15:00:00,1,error4
4,2015-01-22 10:00:00,1,error4


First we must observe that the training set, which until now is stored in "df_maquinas", doesn't have time information, only date. In order to merge "error", we first have to count the number of errors per day and treat the column "error" getting its dummies.

In [14]:
# group erros per day
erros_day = erros.copy()
erros_day['data'] = erros_day['data'].apply(lambda x: x.date())

erros_day = pd.get_dummies(erros_day, columns=['erro'], prefix='', prefix_sep='')
erros_day = erros_day.groupby(['data', 'maquina'], as_index=False).sum()

In [15]:
erros_day.head()

,data,maquina,error1,error2,error3,error4,error5
0,2015-01-01,10,1,0,0,0,0
1,2015-01-01,14,0,0,0,1,0
2,2015-01-01,24,1,0,0,0,0
3,2015-01-01,42,0,1,0,0,0
4,2015-01-01,43,0,0,1,0,0


Now we're able to merge "erros" with "df_maquinas" using "maquina" as the reference column.

In [16]:
# merge dados with erros
merge = pd.merge(dados, erros_day, on='maquina', suffixes=['', '_erro'])
merge['data'] = pd.to_datetime(merge['data'])
merge['data_erro'] = pd.to_datetime(merge['data_erro'])

In [17]:
merge.head()

,index,maquina,data,falha,data_erro,error1,error2,error3,error4,error5
0,20012,55,2015-08-10,ok,2015-01-14,0,0,0,0,1
1,20012,55,2015-08-10,ok,2015-01-18,0,0,1,0,0
2,20012,55,2015-08-10,ok,2015-01-19,1,0,0,0,0
3,20012,55,2015-08-10,ok,2015-01-31,0,1,0,0,0
4,20012,55,2015-08-10,ok,2015-03-04,0,0,1,0,0


In [18]:
# indexes that don't satisfy the window condition -> errors shouldn't be considered (turn into 0)
idx = ((merge['data'] - merge['data_erro']) > window )  | \
       (merge['data'] < merge['data_erro'])
idx = merge[idx].index
merge.loc[idx, 'error1':'error5'] = 0 

In [19]:
# Sum the number of errors within the considered window 
cols = merge.loc[:, 'error1':'error5'].columns
group = merge.groupby('index', as_index=False)
df_erros = group[cols].sum()

In [20]:
df_erros.head()

,index,error1,error2,error3,error4,error5
0,0,0,0,0,0,0
1,3,0,0,0,0,1
2,5,0,0,0,0,0
3,6,0,0,0,0,0
4,7,0,0,0,0,0


Now the index column will be our reference for future merges.

### Merge dados with info_uso => df_info_uso

In [22]:
info_uso.head()

,data,maquina,voltagem,rotacao,pressao,vibracao
0,2015-01-01 06:00:00,1,176.217853,418.504078,113.077935,45.087686
1,2015-01-01 07:00:00,1,162.879223,402.747490,95.460525,43.413973
2,2015-01-01 08:00:00,1,170.989902,527.349825,75.237905,34.178847
3,2015-01-01 09:00:00,1,162.462833,346.149335,109.248561,41.122144
4,2015-01-01 10:00:00,1,157.610021,435.376873,111.886648,25.990511


Again, we have the same problem of features values per hour. We'll proceed with the same "window" approach to calculate mean and std for these features within the window period.

In [23]:
# Split column in date and hour
info_uso_day = info_uso.copy()
info_uso_day['data'] = info_uso_day['data'].apply(lambda x: x.date())
info_uso_day['data'] = pd.to_datetime(info_uso_day['data'])

In [24]:
# Group by day
cols = info_uso_day.loc[:, 'voltagem':'vibracao'].columns
group = info_uso_day.groupby(['maquina', 'data'])[cols]

# Feature engineering
mean = group.mean().rename( columns={col:'mean_'+col for col in cols} )
std = group.std().rename( columns={col:'std_'+col for col in cols} )

info_uso_day = pd.concat([mean, std], axis=1).reset_index()

In [25]:
# Merge train and info_uso grouped by day
merge = pd.merge(dados, info_uso_day, on='maquina', suffixes=['', '_info'])

In [26]:
# Set NaN for "future" date values or values outside the window
idx = ((merge['data'] - merge['data_info']) > window )  | \
       (merge['data'] < merge['data_info'])
idx = merge[idx].index
merge.loc[idx, 'mean_voltagem':] = np.nan

In [27]:
cols = merge.columns[4:] # 'mean_voltagem' : 'std_vibracao'

group = merge.groupby('index', as_index=False)

df_info_uso = group[cols].agg(np.nanmean)

Now let's get together all the information we have until now !

### Merge df_maquinas, df_erros, df_info_uso => df_data 

In [28]:
data = pd.merge(df_maquinas, df_erros, on='index')
data = pd.merge(data, df_info_uso, on='index')

### Dummies of modelo and drop maquina, data columns

In [29]:
# dummie modelo
dumm = pd.get_dummies(data['modelo'])
data = pd.concat([data, dumm], axis=1).drop('modelo', axis=1)

In [30]:
# remove maquina and data columns
data.drop(['maquina', 'data'], axis=1, inplace=True)

if data_type == 'treino':
    data.drop('index', axis=1, inplace=True)

In [31]:
data.head()

,falha,idade,error1,error2,error3,error4,error5,mean_voltagem,mean_rotacao,mean_pressao,mean_vibracao,std_voltagem,std_rotacao,std_pressao,std_vibracao,model1,model2,model3,model4
0,ok,17,0,0,0,0,0,165.678044,436.241599,100.054536,38.053647,15.523593,58.018104,9.715540,4.960047,0,0,1,0
1,ok,17,0,0,0,0,0,167.984429,449.209447,100.395188,40.178632,16.523830,50.504585,12.082691,3.608012,0,0,1,0
2,ok,17,0,0,0,0,0,173.296031,461.113762,96.854299,39.133315,13.776066,43.845024,7.614312,6.009319,0,0,1,0
3,ok,17,0,0,0,0,0,167.224425,450.492274,99.447270,40.829714,18.396645,53.864923,11.908198,5.874302,0,0,1,0
4,ok,17,1,0,0,0,0,173.641802,441.383124,98.744748,39.439531,12.846186,60.070156,7.903329,5.866805,0,0,1,0


In [32]:
# save to csv
days = days.replace(' ', '')
data.to_csv('data_%s_%s.csv' % (data_type, days), index=False)